In [91]:
import pickle
import pandas as pd
import numpy as np
import sys
from IPython.display import display, clear_output
from datetime import datetime
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import os

Филигранно распикливаем ([с возможной утечкой памяти:)](https://stackoverflow.com/questions/7395542/is-explicitly-closing-files-important))

In [2]:
players_dict = pickle.load(open('data/chgk/players.pkl', 'rb'))
results_dict = pickle.load(open('data/chgk/results.pkl', 'rb'))
tournaments_dict = pickle.load(open('data/chgk/tournaments.pkl', 'rb'))

In [3]:
tournaments_dict_train = {k: v for k, v in tournaments_dict.items() if v['dateStart'].startswith('2019')}
tournaments_dict_test = {k: v for k, v in tournaments_dict.items() if v['dateStart'].startswith('2020')}

`players` и `tournaments` сразу в датафрейм, `results` пока что в словарь датафреймов 

In [4]:
results_dict_train = {k: v for k, v in results_dict.items() if k in tournaments_dict_train.keys()}
results_dict_test = {k: v for k, v in results_dict.items() if k in tournaments_dict_test.keys()}

Фильтруем `results`, оставляем только те, где есть `mask`

In [5]:
results_filtered_train = {}
results_filtered_test = {}
for k, res in results_dict.items():
    t_res_new = []
    for t_res in results_dict[k]:
        if 'mask' in t_res.keys():
            if t_res['mask'] is not None:
                if 'X' not in t_res['mask'] and '?' not in t_res['mask']:
                    t_res_new.append(t_res)
    if t_res_new:
        if tournaments_dict[k]['dateStart'].startswith('2019'):
            results_filtered_train[k] = t_res_new
        if tournaments_dict[k]['dateStart'].startswith('2020'):
            results_filtered_test[k] = t_res_new

In [8]:
tournament_question_count = {}

for k, v in results_filtered_train.items():
    tournament_question_count[k]= max([len(t_res['mask']) for t_res in v])
    
for k, v in results_filtered_test.items():
    tournament_question_count[k]= max([len(t_res['mask']) for t_res in v])

In [62]:
results_filtered_train_len = {}
results_filtered_test_len = {}
for k, v in results_filtered_train.items():
    t_res_ = []
    for t_res in v:
        if len(t_res['mask']) == tournament_question_count[k]:
            t_res_.append(t_res)
    results_filtered_train_len[k] = t_res_

for k, v in results_filtered_test.items():
    t_res_ = []
    for t_res in v:
        if len(t_res['mask']) == tournament_question_count[k]:
            t_res_.append(t_res)
    results_filtered_test_len[k] = t_res_

In [64]:
pq_df = pd.DataFrame(columns=['pid', 'qid', 'res'])

In [70]:
pid = []
qid = []
res = []
for k, v in results_filtered_train_len.items():
    for t_res in v:
        members = [m['player']['id'] for m in t_res['teamMembers']]
        pid.extend(list(itertools.chain.from_iterable(itertools.repeat(m, tournament_question_count[k]) for m in members)))
        qid.extend([f'{k}_{i}' for i in range(tournament_question_count[k])] * len(members))
        res.extend(list(map(int, t_res['mask'])) * len(members))

In [71]:
pq_df['pid'] = pid
pq_df['qid'] = qid
pq_df['res'] = res

In [73]:
np.int8

numpy.int8

In [74]:
encoder = OneHotEncoder(categorical_features=[0, 1], sparse=True, dtype=np.int8)

In [75]:
encoder.fit(pq_df)

/Users/a18570724/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=[0, 1], categories=None, drop=None,
              dtype=<class 'numpy.int8'>, handle_unknown='error', n_values=None,
              sparse=True)

In [88]:
pq_df_oh = encoder.transform(pq_df[['pid', 'qid']])

In [93]:
clf = LogisticRegression(n_jobs = 10)

In [94]:
clf.fit(pq_df_oh, pq_df['res'])

/Users/a18570724/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/a18570724/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=10, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

86144